In [1]:
%matplotlib inline
%timeit

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import time
import pickle
import datetime
import re
import pyActigraphy

import sys
sys.path.append('/home/ngrav/project/')
from wearables.scripts import utils as wearutils
from wearables.scripts import data as weardata
from wearables.scripts import train as weartrain
from wearables.scripts import eval_ as weareval
from wearables.scripts import model as wearmodels
from wearables.scripts import DTW as weardtw

import torch
import torch.nn as nn
import torch.nn.functional as F

from scipy.spatial.distance import pdist, squareform
import fastdtw
import umap

plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=1
plt.rcParams['savefig.dpi'] = 600
sns.set_style("ticks")

In [2]:
# load data
mfp = '/home/ngrav/scratch/wearables_model_zoo'
bst_modelpkl = os.path.join(mfp, '281-itv71_InceptionTime_GA4.pkl')
bst_trainer = os.path.join(mfp, 'trainer_itv71_InceptionTime_GA4.pkl')

split = 'train'

X, md_model, trainer = weardtw.data_from_trainer(bst_trainer, split=split)

# call eval to get output, modify for return_hidden = True option
evaluation = weareval.eval_trained(trainer, split=split,
                                   modelpkl=bst_modelpkl,
                                   two_outputs=True)

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/home/ngrav/miniconda3/envs/wearables/lib/python3.8/site-packages/torch/nn/modules/conv.py:294: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/aten/src/ATen/native/Convolution.cpp:660.)
  return F.conv1d(input, weight, bias, self.stride,
/home/ngrav/miniconda3/envs/wearables/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [4]:
md_model

,record_id,age_enroll,marital,gestage_by,insur,ethnicity,race,bmi_1vis,prior_ptb_all,fullterm_births,...,edinb2_1trim,edinb3_1trim,edinb4_1trim,edinb5_1trim,edinb6_1trim,edinb7_1trim,edinb8_1trim,edinb9_1trim,edinb10_1trim,GA
1062_8,1062,31.0,0.0,1.0,5.0,0.0,1.0,40.106270,0.0,0.0,...,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,8.0
2344_32,2344,26.0,1.0,3.0,3.0,0.0,0.0,25.969303,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,32.0
1018_33,1018,21.0,0.0,1.0,1.0,0.0,1.0,23.099243,0.0,1.0,...,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,33.0
2141_31,2141,30.0,1.0,2.0,3.0,0.0,0.0,22.029270,0.0,1.0,...,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,31.0
2037_24,2037,35.0,1.0,3.0,3.0,0.0,0.0,22.994972,0.0,0.0,...,1.0,1.0,3.0,2.0,1.0,1.0,1.0,2.0,1.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2109_8,2109,33.0,1.0,1.0,3.0,0.0,0.0,29.846411,1.0,1.0,...,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,8.0
1488_9,1488,19.0,0.0,1.0,5.0,0.0,1.0,32.438705,0.0,1.0,...,1.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,9.0
2524_33,2524,26.0,1.0,2.0,3.0,0.0,0.0,36.645576,0.0,1.0,...,1.0,1.0,2.0,1.0,3.0,1.0,1.0,1.0,1.0,33.0
1398_23,1398,28.0,0.0,1.0,5.0,0.0,1.0,52.861694,0.0,0.0,...,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,23.0
